# Building Comprehensive AWS Index

The AWS index files are not comprehensive, so if we want to match organizations to specific tax returns or have a comprehensive view of the space, we have to do this more manually.

Before this notebook I have 1) downloaded the current AWS index files from 2015-2019 from https://s3.amazonaws.com/irs-form-990/index_20[XX].csv and 2) used the script from here: https://appliednonprofitresearch.com/posts/2020/06/skip-the-irs-990-efile-indices/ to retrieve a comprehensive list of files. If this is public I will be posting my lightly edited version of that script as well.

This notebook:
1. Chunks the comprehensive file list by year
2. Retrieves basic info from 990s tht are missing in the AWS index
3. Adds all the info into a new comprehensive index

In [5]:
import pandas as pd
import re
import time

from irsx.xmlrunner import XMLRunner

## Chunking Comprehensive File List

This just makes everything a bit more manageable.

In [12]:
# Change based on when you want to retrieve
BEG_YR = 2015
END_YR = 2019
yr_lst = list( range( BEG_YR, END_YR + 1 ) )

In [ ]:
# First doing some managing of the result of the script by separating it into years
# complete_file_list.csv is simply a one column (called "file_name") list.
# This notebook assumes and saves all files in the local directory
file_list = pd.read_csv( "complete_file_list.csv" )

for yr in yr_list:
    save_filename = "file_list_" + str( yr ) + ".csv"
    file_list[ file_list['file_name'].str.startswith( str( yr ) ) ].to_csv( save_filename, index=False )

file_list = None

## Retrieve 990 Information

I'm only going to retrieve the interpretable information that already exists in the AWS index files, that is, EIN, Name, and Form. As the above linked blog post points out, TAX PERIOD is unreliable. My guess is that if you're using this, you don't need to know tax period.

Using IRSx for retrieval.

This takes a very long time (on the order of days). Luckily I had other things I was doing this week, but hopefully will never have to do 6 years at a time again.

In [ ]:
start_time = time.time()
update_period = 5000 # How often you want to be updated about progress.

for yr in yr_lst:
    
    str_yr = str( yr )
    print( "Starting with " + str_yr + " at " + str( round( time.time() - start_time, 1 ) ) + " seconds." )
    
    # Get comprehensive list of files for the year and create object ID from the file
    file_lst_df = pd.read_csv( "file_list_" + str_yr + ".csv" )
    file_lst_df['OBJECT_ID'] = file_lst_df['file_name'].str[:18].astype( int )
    
    # Get AWS index to focus only on files that are not covered
    aws_ind = pd.read_csv( "index_" + str_yr + ".csv" )
    file_lst_df = file_lst_df[~file_lst_df['OBJECT_ID'].isin( aws_ind['OBJECT_ID'] )]
    
    print( str_yr + " list has " + str( len( file_lst_df ) ) + " missing." )
    
    # Iterate through, adding information to lists
    xml_runner = XMLRunner()
    yr_oid = []
    yr_ein = []
    yr_name = []
    yr_form = []
    counter = 0
    for index, row in file_lst_df.iterrows():
        
        # Retrieve 990
        try:
            header_dat = xml_runner.run_sked( row['OBJECT_ID'], 'ReturnHeader990x').get_result()
        except:
            header_dat = None
            
        # Read in applicable data
        try:
            file_ein = header_dat[0]['schedule_parts']['returnheader990x_part_i']['ein']
        except:
            file_ein = 0
        try:
            file_name1 = header_dat[0]['schedule_parts']['returnheader990x_part_i']['BsnssNm_BsnssNmLn1Txt']
        except:
            file_name1 = ''
        try:
            file_name2 = header_dat[0]['schedule_parts']['returnheader990x_part_i']['BsnssNm_BsnssNmLn2Txt']
        except:
            file_name2 = ''
        file_name_c = ( file_name1 + ' ' + file_name2 ).strip().lower()
        try:
            file_form = header_dat[0]['schedule_parts']['returnheader990x_part_i']['RtrnHdr_RtrnCd']
        except:
            file_form = ''

        yr_oid.append( str( row['OBJECT_ID'] ) )
        yr_ein.append( file_ein )
        yr_name.append( file_name_c )
        yr_form.append( file_form )
        
        # Update me about progress - one thing I did not do but might want to try is adding a pause in here
        # the retrieval is always faster after a break.
        if ( counter % update_period ) == 0:
            print( str_yr + " counter at: " + str( counter ) + " out of " + str( len( file_lst_df ) ) +\
                   " (" + str( round( counter / len( file_lst_df ), 2 ) ) + ")" +\
                   " at " + str( round( time.time() - start_time, 1 ) ) + " seconds." )
        counter += 1
    
    # Save file as a csv from a pandas dataframe
    pd.DataFrame( {'OBJECT_ID': yr_oid, 'EIN': yr_ein, 'TAXPAYER_NAME': yr_name, 'RETURN_TYPE': yr_form} ).\
        to_csv( str_yr + "_addit_index.csv" )

## Build Comprehensive Index

This step could have been easily combined into the iteration above, but it's nice to be a bit chunked.

In [14]:
for yr in yr_lst:
    aws_ind = pd.read_csv( "index_" + str( yr ) + ".csv" )
    aws_ind['INDEX_SRC'] = "AWS INDEX"
    new_ind = pd.read_csv( str( yr ) + "_addit_index.csv", index_col=0 )
    new_ind['INDEX_SRC'] = "AWS FILE DIR"
    new_ind['TAXPAYER_NAME'] = new_ind['TAXPAYER_NAME'].str.upper()
    aws_ind.append( new_ind ).to_csv( "all_file_index_" + str( yr ) + ".csv", index=False )
    print( str( yr ) + " AWS index file missing " +\
           str( round( 100 * len( new_ind ) / ( len( new_ind ) + len( aws_ind ) ), 2 ) ) +\
           "% of filings." )

2015 AWS index file missing 31.16% of filings.
2016 AWS index file missing 28.41% of filings.
2017 AWS index file missing 15.56% of filings.
2018 AWS index file missing 16.2% of filings.
2019 AWS index file missing 26.08% of filings.
